#### Все блоки под комментариями являются экспериментами (за частую не удачными)

### После более 50 итераций ноутбука (включая локальные прогоны) решил остановиться на этой версии, показатели которой чуть ниже лучшего варианта, но зато оформленно не как скетч (в сравнении конечно же)

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For examplfe, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import re #regex for comfort
import requests #for parsing


# импортируем библиотеки для визуализации
import matplotlib.pyplot as plt
import matplotlib
import seaborn as sns 
%matplotlib inline
import seaborn as sns

from sklearn.metrics import matthews_corrcoef # для рассмотрения корреляции бинарных признаков
from scipy import stats #для определения распределений

# Загружаем специальный удобный инструмент для разделения датасета:
from sklearn.model_selection import train_test_split

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
        

from geopy.distance import geodesic
from geopy.geocoders import Nominatim

from bs4 import BeautifulSoup
import time

#pd.set_option('max_columns', None)
#pd.set_option('max_rows', None)
pd.set_option('display.max_colwidth', None)
# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
# всегда фиксируйте RANDOM_SEED, чтобы ваши эксперименты были воспроизводимы!
RANDOM_SEED = 42

In [ ]:
hotels = pd.read_csv('./data/hotels.csv')
hotels.head(3)

In [ ]:
hotels.info()

In [ ]:
hotels = hotels.drop_duplicates()

---

### Посмотрим распределение первичных признаков

In [ ]:
fig, (ax1) = plt.subplots(ncols=3, nrows=2, figsize=(15, 10))
sns.kdeplot(hotels['reviewer_score'], ax=ax1[0,0], label ='reviewer_score')
sns.kdeplot(hotels['review_total_negative_word_counts'], ax=ax1[0,1], label ='review_total_negative_word_counts')
sns.kdeplot(hotels['review_total_positive_word_counts'], ax=ax1[0,2], label ='review_total_positive_word_counts')

sns.kdeplot(hotels['additional_number_of_scoring'], ax=ax1[1,0], label ='additional_number_of_scoring')
sns.kdeplot(hotels['average_score'], ax=ax1[1,1], label ='average_score')
sns.kdeplot(hotels['total_number_of_reviews'], ax=ax1[1,2], label ='total_number_of_reviews')
;

### Наблюдается картина близкая к следующим распределениям:
#### - reviewer_score: бета-распределение
#### - review_total_negative_word_counts: экспоненциальное распределение
#### - review_total_positive_word_counts: экспоненциальное распределение
#### - additional_number_of_scoring: экспоненциальное распределение
#### - average_score: бета (до этого строил qq plot)
#### - total_number_of_reviews: экспоненциальное распределение

#### Забегая вперед, попытки выправить минмакс и робуст скаллерами либо ухудшали, либо оставляли неизменными результаты, подозреваю, что либо были выбраны не те фиты, либо количество шума в модели было запредельным

---

In [ ]:
hotels.head(5)

In [ ]:
#df_test.info()

In [ ]:
#df_test.head(2)

In [ ]:
#sample_submission.head(2)

In [ ]:
s#ample_submission.info()

In [ ]:
data = hotels

---

### Очистка признаков (в последствии было принято решение не чистить на выбросы, ибо ухудшалась оценка, скорее всего это связано с тем, что предельно низкие оценки в данном случае являются спецификой)

In [ ]:
# Функция z-отклонений

def outliers_z_score(data, feature, left_mod=2, right_mod=3, log_scale=False):
    
    if log_scale:
        x = np.log(data[feature]+1)
    else:
        x = data[feature]

    mu = x.mean()

    sigma = x.std()

    lower_bound = mu - left_mod * sigma

    upper_bound = mu + right_mod * sigma

    outliers = data[(x < lower_bound) | (x > upper_bound)]

    cleaned = data[(x > lower_bound) & (x < upper_bound)]

    print(f'Количество выбросов по z-методу: {outliers.shape[0]}')
    print(f'Результирующее количество строк в очищенном фрейме: {cleaned.shape[0]}')

    return outliers, cleaned

In [ ]:
# Выбросы в оценках
#sns.boxplot(hotels['reviewer_score'])

In [ ]:
#outliers, data = outliers_z_score(data=hotels, feature='reviewer_score', log_scale=True, left_mod=3.5, right_mod=3.5)

In [ ]:
# Проверка выбросов в графе количество негативных слов
#sns.boxplot(hotels['review_total_negative_word_counts'])

In [ ]:
#outliers, data = outliers_z_score(data=hotels, feature='review_total_negative_word_counts', log_scale=True, left_mod=3.5, right_mod=3.5)

In [ ]:
#outliers, data = outliers_z_score(data=hotels, feature='average_score', log_scale=True, left_mod=3.5, right_mod=3.5)

---

In [ ]:
# ВАЖНО! дря корректной обработки признаков объединяем трейн и тест в один датасет
data['sample'] = 1 # помечаем где у нас трейн
df_test['sample'] = 0 # помечаем где у нас тест
df_test['reviewer_score'] = 0 # в тесте у нас нет значения reviewer_score, мы его должны предсказать, по этому пока просто заполняем нулями

data = df_test.append(data, sort=False).reset_index(drop=True) # объединяем

In [ ]:
data.info()

In [ ]:
fig, (ax1) = plt.subplots(ncols=3, nrows=2, figsize=(15, 10))
sns.kdeplot(data['reviewer_score'], ax=ax1[0,0], label ='reviewer_score')
sns.kdeplot(data['review_total_negative_word_counts'], ax=ax1[0,1], label ='review_total_negative_word_counts')
sns.kdeplot(data['review_total_positive_word_counts'], ax=ax1[0,2], label ='review_total_positive_word_counts')

sns.kdeplot(data['additional_number_of_scoring'], ax=ax1[1,0], label ='additional_number_of_scoring')
sns.kdeplot(data['average_score'], ax=ax1[1,1], label ='average_score')
sns.kdeplot(data['total_number_of_reviews'], ax=ax1[1,2], label ='total_number_of_reviews')
;

### Из-за добавления нулевого столбца из тестового сета review_score приобрел характерный пик

---

### Преобразование тегов (в датасете теги представлены единой строкой)

In [ ]:
# Пркобразование тегов к списку
def tag_reconstr(data):
    temp_list = []
    for i in data.split("'"):
        temp_list.append(i.strip())

    temp_list.remove('[')
    temp_list.remove(']')

    for i in temp_list:
        if i == ',':
            temp_list.remove(i)
    return temp_list


tags_series = data['tags'].apply(tag_reconstr)
data['tags'] = data['tags'].apply(tag_reconstr)


In [ ]:
#Формируем базу для one_hot, параллельно выдергиваем фит прожитых ночей
all_tags = tags_series.explode()
stayed_night_mask = all_tags.str.contains('Stayed', regex=False)
stayed_night = all_tags[stayed_night_mask]
stayed_night.str.split(' ')

def night_at_hotel(data):
    temp_var = 0
    for i in data.split(' '):
        try:
            temp_var = int(i)
        except ValueError:
            pass
    return temp_var

nights = stayed_night.apply(night_at_hotel)
nights = nights.rename('nights')
data = pd.concat([data, nights], axis=1)
data['nights'] = data['nights'].fillna(data['nights'].mode()[0])

In [ ]:
# Посмотрим топ-30 тагов без учета ночей
all_tags[~stayed_night_mask].value_counts()[:30]

In [ ]:
# Кодируем тип путешествия
traveller_types = {
    'Solo traveler': 1,
    'Couple': 2,
    'Family with young children': 3,
    'Family with older children': 4,
    'Group': 5
}

In [ ]:
room_types = [
    'without Window', 'Guestroom', 'Classic', 'Single', 
    'Standard', 'Superior', 'Comfort', 'Club', 'Suite', 
    'Deluxe', 'King', 'Premier', 'Deluxe Double Room', 
    'Double or Twin Room', 'Double Room', 'Standard Double or Twin Room',
    'Standard Double Room', 'Superior Double Room']


### На данном этапе было выявлено положительное влияние onehot кодирования тегов, экспериментально была найдена планка оптимального количества, в контексте использованных мной подходов

In [ ]:
#Вытаскиваем топ 40 тегов исключая количество ночей и разворачиваем на их основе one_hot
top_tags = all_tags.value_counts()[:40]

one_hot_columns = []
for i in top_tags.index:
    #if i in room_types or i.startswith('Stayed'):
    if i.startswith('Stayed'):
        pass
    else:
        one_hot_columns.append(i)


    
'''# One hot для типа номера
for room in room_types:
    data[room] = data['tags'].apply(lambda x: 1 if room in x else 0)'''


In [ ]:
for element in one_hot_columns:
    data[element] = data['tags'].apply(lambda x: 1 if element in x else 0)

In [ ]:
data.info()

---

### Кодируем отели

In [ ]:
#Кодируем названия отелей
data['hotel_name'] = data['hotel_name'].astype('category')
data['hotel_name'] = data['hotel_name'].cat.codes

In [ ]:
data.head()

### Код отеля по коду его страны

#### kaggle не работает с requests потому пришлось тянуть информацию локально, так же была попытка стянуть информацию с рейтинга отеля представленного в гугл используя beautifulsoup, но столкнулся с проблемой постоянного бана со стороны гугл, из ~1490 уникальных отелей удалось получить информацию лишь о 1000, и, проанализировав ситуацию, и сделав вывод что рейтин отеля будет коррелировать с кодом отеля и от одного признака все равно придется избавляться, решил оставить код

In [ ]:
'''# Достаем код страны
response = requests.get('https://countrycode.org/')
country_code = pd.read_html(response.text)
country_code = country_code[0]
country_code = country_code[['COUNTRY', 'COUNTRY CODE']]

# Создаем функцию для формирования кодовой строки исходя из принципа территориальной близости принадлежности для кодов формата Х-ХХХ
def get_code(data):
    data = data.split('-')
    return int(data[0])


country_code['COUNTRY CODE'] = country_code['COUNTRY CODE'].apply(get_code)'''

country_code = pd.read_csv('/kaggle/input/country-code-2/country_code.csv')

In [ ]:
# Формируем новый закодированный (в последствие) признак местоположения отеля
data['hotel_country'] = data['hotel_address'].str.split(' ')
data['hotel_country'] = data['hotel_country'].apply(lambda x: x[-2::])
data['hotel_country'] = data['hotel_country'].apply(
    lambda x: x[0]+' '+x[1] if x[0] == 'United' else x[1])

data = data.merge(
    country_code, how='left',
    left_on='hotel_country', right_on='COUNTRY'
)
data = data.rename(columns={'COUNTRY CODE': 'hotel_country_code'})

---

### Код ревьюера по его национальной принадлежности

In [ ]:
# Стрипуем национальность для будущего присвоения кода страны
data['reviewer_nationality'] = data['reviewer_nationality'].str.strip()
# Переименуем пару стран
data['reviewer_nationality'] = data['reviewer_nationality'].str.replace(
    'United States of America', 'United States')

# Формируем новый фрейм с кодом страны ревьюера
data = data.merge(
    country_code, how='left',
    left_on='reviewer_nationality', right_on='COUNTRY'
)

data = data.rename(
    columns={'COUNTRY CODE': 'nationality_code', 'COUNTRY': 'reviewer_country'})

data['nationality_code'] = data['nationality_code'].fillna(
    data['nationality_code'].mode()[0])


### Являетлся ли ревьюер жителем страны в которой находится отель

In [ ]:
data['is_citizen'] = data['nationality_code'] == data['hotel_country_code']
data['is_citizen'] = data['is_citizen'].apply(lambda x: 1 if x == True else 0)

### Выкинем колонки которые больше не будут участвовать в анализе

In [ ]:
data.drop(['COUNTRY_x', 'COUNTRY_y'], axis=1, inplace=True)

---

### Подсмотрел у коллег модуль для семантического анализа отзывов, т.к. собственные изыскания приводили к использованию сложных для меня на данном этапе инструментов

In [ ]:
from afinn import Afinn
afinn = Afinn()

In [ ]:
# Стрипуем озоры
data['negative_review'] = data['negative_review'].str.strip()
data['positive_review'] = data['positive_review'].str.strip()

### Приведем в порядок тексты обзоров и пересчитаем количество слов

In [ ]:
data['negative_review'] = data['negative_review'].apply(
    lambda x: '' if x == 'No Negative' or x == 'Nothing' 
    or x == 'nothing' or x == 'None' or x == 'N A' 
    or x == 'Nothing really' or x == 'N a' or x == 'All good' 
    or x == 'No complaints' or x == 'Nothing at all' or x == 'Nothing to dislike' or x == 'none' else x)


In [ ]:
data['review_negative_word_counts'] = data['negative_review'].str.split(' ')

In [ ]:
data['review_negative_word_counts'] = data['review_negative_word_counts'].apply(lambda x: len(x))

In [ ]:
data.drop(['review_total_negative_word_counts'], axis=1, inplace=True)

___

In [ ]:
data['positive_review'] = data['positive_review'].apply(
    lambda x: '' if x == 'No Positive' or x == 'Nothing' else x)

In [ ]:
data['review_positive_word_counts'] = data['positive_review'].str.split(' ')

In [ ]:
data['review_positive_word_counts'] = data['review_positive_word_counts'].apply(lambda x: len(x))

In [ ]:
data.drop(['review_total_positive_word_counts'], axis=1, inplace=True)

___

### На данном этапе семантический анализ показал низкие результаты, тем не менее, даже это дает прибавку к конечному результату.

In [ ]:
# Создадим фит общего ревью для sentiment анализа общей картины ревью
#data['whole_review'] = data['negative_review'] + ' ' + data['positive_review']

In [ ]:
# Проведем семантический анализ ревью
data['semantic_analisis_neg'] = data['negative_review'].apply(lambda x: afinn.score(x))
data['semantic_analisis_pos'] = data['positive_review'].apply(lambda x: afinn.score(x))
#data['semantic_analisis_whole'] = data['whole_review'].apply(lambda x: afinn.score(x))

In [ ]:
#test_frame = data.query("sample == 1")[['whole_review','semantic_analisis_whole','reviewer_score']]
#test_frame.query("semantic_analisis_whole > 0 and reviewer_score < 7")[0:5]

---

### Вытаскиваем город для дальнейших манипуляций с растояниями до различных объектов

In [ ]:
#Создаем признак города
data['city'] = data['hotel_address'].str.split(' ')
data['city'] = data['city'].apply(lambda x: x[-5] if x[-1] == 'Kingdom' else x[-2])

In [ ]:
#Cоздаем список городов в сете
city_list = list(data['city'].unique())
city_list

In [ ]:
# Разворачиваем one_hot
for city in city_list:
    data[city] = data['city'].apply(lambda x: 1 if city in x else 0)

In [ ]:
# Условные центры городов выбранные на основание исторических достопримечательностей. 
# В данном случае рассматриваются центры притяжения для отдыха, но не бизнес поездок, 
# т.к. предыдущие изыскания показали превалирующую долю в датасете и признак бизнес поездок был дропнут
# в виду наличия мультиколлинеарности и меньшей доли. 

cities_coordinates = {
    'London': (51.50458, -0.1332), 'Paris': (48.85637, 2.33475),
    'Amsterdam': (52.374787, 4.896339), 'Milan': (45.470098, 9.179906),
    'Vienna': (48.207904, 16.368116), 'Barcelona': (41.399367, 2.169938)
}


### Немного прервемся с расстоянием

### Заполняем пропуски в широте и долготе используя адреса отелей

In [ ]:
'''#Создаем пустые списки, в них будем нарезать куски адресов, точность будет не абсолюьная, но приемлимая
temp_list = []
list_of_nan = []
address_list = []

#Создаем список уникальных адресов отелей с пропусками
empty_loc = data[data['lat'].isnull()]
empty_hot_list = list(empty_loc['hotel_address'].unique())

#Планомерно заполняем наши списки, впоследующем address_list используем для получения кортежей lat/lng
for i in empty_hot_list:
    temp_var = i.split(' ')[-1:-4:-1]
    temp_list.append(temp_var)
    
for i in temp_list:
    list_of_nan.append(i[::-1])

for i in list_of_nan:
    address_list.append(' '.join(i))
    

#Пробегаем циклом по адресам и создаем список кортежей ключ- часть адреса, значение lat/lng
location_tuple_dict = dict()
lat_dict = dict()
lng_dict = dict()
geolocator = Nominatim(user_agent="my_request")
for loc in address_list:
    location = geolocator.geocode(loc)
    location_tuple_dict[loc]=(location.latitude, location.longitude)
    lat_dict[loc] = location.latitude
    lng_dict[loc] = location.longitude

# Востанавливаем словарь с полным названием, после всех манипуляций у нас есть словарь с ключами
# в виде полного адреса и значениями в виде кортежа координат
new_lat_dict = dict()
for i in lat_dict:
    for j in empty_hot_list:
        if i in j:
            new_lat_dict[j] = lat_dict[i]
            
new_lng_dict = dict()
for i in lng_dict:
    for j in empty_hot_list:
        if i in j:
            new_lng_dict[j] = lng_dict[i]'''

In [ ]:
# Это результат предыдущего кода, который здесь не выполним
new_lat_dict = {
    '23 Rue Damr mont 18th arr 75018 Paris France': 48.88948513718876,
    'Josefst dter Stra e 10 12 08 Josefstadt 1080 Vienna Austria': 48.21102906492974,
    'Josefst dter Stra e 22 08 Josefstadt 1080 Vienna Austria': 48.21102906492974,
    'W hringer Stra e 33 35 09 Alsergrund 1090 Vienna Austria': 48.22275737819706,
    'W hringer Stra e 12 09 Alsergrund 1090 Vienna Austria': 48.22275737819706,
    'Gr nentorgasse 30 09 Alsergrund 1090 Vienna Austria': 48.22275737819706,
    'Sieveringer Stra e 4 19 D bling 1190 Vienna Austria': 48.24951395649527,
    'Hasenauerstra e 12 19 D bling 1190 Vienna Austria': 48.24951395649527,
    'Taborstra e 8 A 02 Leopoldstadt 1020 Vienna Austria': 48.214452907946104,
    'Bail n 4 6 Eixample 08010 Barcelona Spain': 41.39148971786492,
    'Landstra er G rtel 5 03 Landstra e 1030 Vienna Austria': 48.19869400654681,
    'Paragonstra e 1 11 Simmering 1110 Vienna Austria': 48.166512303667986,
    'Savoyenstra e 2 16 Ottakring 1160 Vienna Austria': 48.212691796580266,
    '20 Rue De La Ga t 14th arr 75014 Paris France': 48.83082152471132,
    '4 rue de la P pini re 8th arr 75008 Paris France': 48.87375053595041,
    'Sep lveda 180 Eixample 08011 Barcelona Spain': 41.38439987783505,
    'Pau Clar s 122 Eixample 08009 Barcelona Spain': 41.39475249171975
}

new_lng_dict = {
    '23 Rue Damr mont 18th arr 75018 Paris France': 2.342177494859438,
    'Josefst dter Stra e 10 12 08 Josefstadt 1080 Vienna Austria': 16.347425397892273,
    'Josefst dter Stra e 22 08 Josefstadt 1080 Vienna Austria': 16.347425397892273,
    'W hringer Stra e 33 35 09 Alsergrund 1090 Vienna Austria': 16.35633361838874,
    'W hringer Stra e 12 09 Alsergrund 1090 Vienna Austria': 16.35633361838874,
    'Gr nentorgasse 30 09 Alsergrund 1090 Vienna Austria': 16.35633361838874,
    'Sieveringer Stra e 4 19 D bling 1190 Vienna Austria': 16.33880145212523,
    'Hasenauerstra e 12 19 D bling 1190 Vienna Austria': 16.33880145212523,
    'Taborstra e 8 A 02 Leopoldstadt 1020 Vienna Austria': 16.397041956849197,
    'Bail n 4 6 Eixample 08010 Barcelona Spain': 2.1746231466230936,
    'Landstra er G rtel 5 03 Landstra e 1030 Vienna Austria': 16.393263401622043,
    'Paragonstra e 1 11 Simmering 1110 Vienna Austria': 16.43628858500086,
    'Savoyenstra e 2 16 Ottakring 1160 Vienna Austria': 16.311862581883567,
    '20 Rue De La Ga t 14th arr 75014 Paris France': 2.3248247815242493,
    '4 rue de la P pini re 8th arr 75008 Paris France': 2.314978307438017,
    'Sep lveda 180 Eixample 08011 Barcelona Spain': 2.158941018556701,
    'Pau Clar s 122 Eixample 08009 Barcelona Spain': 2.169325250955414
}


In [ ]:
#Заполним пустоты в широте
data['lat'] = data['lat'].fillna(data['hotel_address'].apply(lambda x: new_lat_dict.get(x)))

In [ ]:
#И долготе
data['lng'] = data['lng'].fillna(data['hotel_address'].apply(lambda x: new_lng_dict.get(x)))

In [ ]:
#Создаем новый фит координат отеля
data['coordinates'] = tuple(zip(data['lat'], data['lng']))

### Вернемся к расстояниям

In [ ]:
data['local_center'] = data['city'].apply(lambda x: cities_coordinates[x])

In [ ]:
# Функция для нахождения расстояния
def distancer(row):
    coords_1 = row['coordinates']
    coords_2 = row['local_center']
    return geodesic(coords_1, coords_2).km

data['distance_center'] = data.apply(distancer, axis=1)

In [ ]:
data['distance_center'].describe()

In [ ]:
sns.boxplot(data['distance_center'])

In [ ]:
# Разведем сеты на время
hotels = data.query('sample == 1')
test_data = data.query('sample == 0')

In [ ]:
outliers, data = outliers_z_score(data=hotels, feature='distance_center', log_scale=True, left_mod=3.5, right_mod=3.5)

In [ ]:
data = test_data.append(hotels, sort=False).reset_index(drop=True) # объединяем обратно

In [ ]:
# Функция определяющая группу удаленности от центра
def distance_group(data):
    if data <= 0.5:
        return 'face_to_face'
    elif 0.5 < data <= 2.5:
        return 'close'
    elif 2.5 < data <= 4:
        return 'middle'
    else:
        return 'far_cry'

In [ ]:
data['distance_group'] = data['distance_center'].apply(distance_group)

In [ ]:
#Удаляем фиты, которые больше не понадобятся
data.drop(['coordinates', 'local_center', 'distance_center', 'lat', 'lng'], axis=1, inplace=True)

In [ ]:
distance_dummies = pd.get_dummies(data['distance_group'])
data = data.merge(distance_dummies, left_index=True, right_index=True)

In [ ]:
data.drop(['distance_group'], axis=1, inplace=True)

---

### Вытаскиваем день недели и отдельно месяц

In [ ]:
data['review_date'] = pd.to_datetime(data['review_date'])
data['month'] = data['review_date'].dt.month
data['day_of_week'] = data['review_date'].dt.day_of_week
data['year'] = data['review_date'].dt.year

In [ ]:
# Попробуем закодировать признак года
data['year'].value_counts()

In [ ]:
years = pd.get_dummies(data['year'], prefix='year')
data = data.merge(years, left_index=True, right_index=True)

In [ ]:
data.drop(['year'], axis=1, inplace=True)

In [ ]:
# Закодируем дни недели
week_days = pd.get_dummies(data['day_of_week'], prefix='day_of_week')
data = data.merge(week_days, left_index=True, right_index=True)

In [ ]:
data.drop(['day_of_week'], axis=1, inplace=True)

In [ ]:
# Закодируем сезоны
def seasons(data):
    if data in [12,1,2]:
        return 'winter'
    elif data in [3,4,5]:
        return 'spring'
    elif data in [6,7,8]:
        return 'summer'
    else:
        return 'autumn'

In [ ]:
data['season'] = data['month'].apply(seasons)

In [ ]:
seasons = pd.get_dummies(data['season'], prefix='season')
data = data.merge(seasons, left_index=True, right_index=True)

In [ ]:
data.drop(['month'], axis=1, inplace=True)

---

### Количество дней с момента отзыва как int

In [ ]:
# Достаем количество дней c момента публикации ревью
def days_num(string_):
    regexp = re.findall(r'\d{0,5}', string_)
    return regexp[0]


data['days_since_review'] = data['days_since_review'].apply(days_num)
data['days_since_review'] = data['days_since_review'].astype('int')


In [ ]:
#Проверим выбросы в разрезе количества дней пройденных с момента отзыва
sns.boxplot(y=data['days_since_review'])

___

### Мультиколлинеарность используем метод Спирмена, т.к. рассмотренные признаки не распределены нормально

In [ ]:
whole_corr = data.corr(method='spearman')
graph_corr = whole_corr[(whole_corr > 0.7) | (whole_corr < -0.7)& (whole_corr != 1)]

sns.heatmap(whole_corr)

In [ ]:
whole_corr[((whole_corr > 0.7) | (whole_corr < -0.7)) & (whole_corr != 1)]

In [ ]:
data_final = data.copy()

In [ ]:
columns_to_drop = ['year_2017','additional_number_of_scoring','Business trip','hotel_country_code']

In [ ]:
data_final.drop(columns_to_drop, axis=1,inplace=True)

---

### Моделирование

In [ ]:
#Дропаем строковые фиты
object_columns = [s for s in data_final.columns if data_final[s].dtypes == 'object']
data_final.drop(object_columns, axis = 1, inplace=True)


In [ ]:
#Дропаем столбец даты что бы модель могла работать
data_final.drop(['review_date'], axis=1, inplace=True)

---

### В очередной раз проверим распределения

In [ ]:
data_final.info()

In [ ]:
fig, (ax1) = plt.subplots(ncols=3, nrows=2, figsize=(15, 10))
sns.kdeplot(data_final['reviewer_score'], ax=ax1[0,0], label ='reviewer_score')
sns.kdeplot(data_final['review_negative_word_counts'], ax=ax1[0,1], label ='review_total_negative_word_counts')
sns.kdeplot(data_final['review_positive_word_counts'], ax=ax1[0,2], label ='review_total_positive_word_counts')

#sns.kdeplot(data_final['additional_number_of_scoring'], ax=ax1[1,0], label ='additional_number_of_scoring') ее дропнули из-за мультиколллинеарности
sns.kdeplot(data_final['average_score'], ax=ax1[1,1], label ='average_score')
sns.kdeplot(data_final['total_number_of_reviews'], ax=ax1[1,2], label ='total_number_of_reviews')
;

### Попробуем нормализовать количество слов в отзыве

In [ ]:
from sklearn import preprocessing

# инициализируем нормализатор MinMaxScaler
mm_scaler = preprocessing.MinMaxScaler()

# кодируем исходный датасет
data_final['review_total_positive_word_counts_norm']  = pd.DataFrame(
    mm_scaler.fit_transform(pd.DataFrame(data_final['review_positive_word_counts'])),
    columns=['review_positive_word_counts'])

data_final['review_total_negative_word_counts_norm']  = pd.DataFrame(
    mm_scaler.fit_transform(pd.DataFrame(data_final['review_negative_word_counts'])),
    columns=['review_negative_word_counts'])


In [ ]:
data_final.drop(['review_negative_word_counts','review_positive_word_counts'],axis=1,inplace=True)

In [ ]:
data_final.head()

In [ ]:
fig, (ax1) = plt.subplots(ncols=3, nrows=2, figsize=(15, 10))

#sns.kdeplot(data_mm['reviewer_score'], ax=ax1[0,0], label ='reviewer_score')
sns.kdeplot(data_final['review_total_positive_word_counts_norm'], ax=ax1[0,1], label ='review_total_positive_word_counts_norm')
sns.kdeplot(data_final['review_total_positive_word_counts_norm'], ax=ax1[0,2], label ='review_total_positive_word_counts_norm')

#sns.kdeplot(data_final['additional_number_of_scoring'], ax=ax1[1,0], label ='additional_number_of_scoring') ее дропнули из-за мультиколллинеарности
#sns.kdeplot(data_mm['average_score'], ax=ax1[1,1], label ='average_score')
#sns.kdeplot(data_mm['total_number_of_reviews'], ax=ax1[1,2], label ='total_number_of_reviews')

___

### Приступаем к моделированию

In [ ]:
# Теперь выделим тестовую часть
train_data = data_final.query('sample == 1').drop(['sample'], axis=1)
test_data = data_final.query('sample == 0').drop(['sample'], axis=1)

y = train_data.reviewer_score.values            # наш таргет
X = train_data.drop(['reviewer_score'], axis=1)

In [ ]:
# Воспользуемся специальной функцие train_test_split для разбивки тестовых данных
# выделим 20% данных на валидацию (параметр test_size)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.15, random_state=RANDOM_SEED)

In [ ]:
# проверяем
test_data.shape, train_data.shape, X.shape, X_train.shape, X_test.shape

In [ ]:
# Импортируем необходимые библиотеки:
from sklearn.ensemble import RandomForestRegressor # инструмент для создания и обучения модели
from sklearn import metrics # инструменты для оценки точности модели

In [ ]:
model = RandomForestRegressor(n_estimators=100, verbose=1, n_jobs=-1, random_state=RANDOM_SEED)
model.fit(X_train, y_train)
y_pred = model.predict(X_test)

In [ ]:
# Сравниваем предсказанные значения (y_pred) с реальными (y_test), и смотрим насколько они в среднем отличаются
# Метрика называется Mean Absolute Error (MAE) и показывает среднее отклонение предсказанных значений от фактических.
print('MAE:', metrics.mean_absolute_error(y_test, y_pred))

MAE: 0.8919320850941495

In [ ]:
def mean_absolute_percentage_error(y_tr, y_pr):
    y_tr, y_pr = np.array(y_tr), np.array(y_pr)
    return np.mean(np.abs((y_tr - y_pr) / y_tr)) * 100

print('MAPE:', mean_absolute_percentage_error(y_test, y_pred))

MAPE: 12.81405005666042

In [ ]:
# в RandomForestRegressor есть возможность вывести самые важные признаки для модели
plt.rcParams['figure.figsize'] = (10,10)
feat_importances = pd.Series(model.feature_importances_, index=X.columns)
feat_importances.nlargest(15).plot(kind='barh')

In [ ]:
test_data.sample(10)

In [ ]:
test_data = test_data.drop(['reviewer_score'], axis=1)

In [ ]:
sample_submission.head()

In [ ]:
predict_submission = model.predict(test_data)

In [ ]:
predict_submission

In [ ]:
list(sample_submission)

In [ ]:
sample_submission['reviewer_score'] = predict_submission
sample_submission.to_csv('submission.csv', index=False)
sample_submission.head(10)